In [1]:
import SkyRunner

env = SkyRunner.CustomEnv()

[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


In [5]:
import train_openAI
import importlib

importlib.reload(train_openAI)

train_openAI.train(env)

9
4
14
1008
-9
5
1001
-5
-4
-4
10
13
-4
13
-10
9
11
-10
15
10
9
4
5
-1
9
0
-1
-8
-10
15
-3
-10
-10
-10
-5
0
-10
9
6
-10
14
-10
15
-4
-10
21
-2
-10
1
5


KeyboardInterrupt: 

In [3]:
env.quit()

env.close()

AttributeError: 'CustomEnv' object has no attribute 'quit'

In [ ]:
env.reset()